In [1]:
import os
if 'models' == os.getcwd().split('/')[-1]: os.chdir('..')
if 'ev_charging' == os.getcwd().split('/')[-1]: print('in the right place!')
else: os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
os.getcwd()

in the right place!


'/home/ewillard/school/capstone/ev_charging'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

import torch
import torch.nn as nn
import math
import time

from src.data_preprocessing import  datetime_processing, userinput_processing, holiday_processing, create_all_site_x, create_all_site_y, create_x, update_varuns_x, get_start_end_times

In [3]:
df_c = pd.read_parquet('data/Sessions/caltech/').reset_index(drop=True)
df_c = datetime_processing(df_c)
df_c = userinput_processing(df_c)

df_j = pd.read_parquet('data/Sessions/jpl/').reset_index(drop=True)
df_j = datetime_processing(df_j)
df_j = userinput_processing(df_j)

df_o = pd.read_parquet('data/Sessions/office001/').reset_index(drop=True)
df_o = datetime_processing(df_o)
df_o = userinput_processing(df_o)

cols = list(df_c.columns)
df = pd.concat([df_c[cols], df_j[cols], df_o[cols]], axis=0)

In [4]:
df.head()

,_id,sessionID,stationID,spaceID,siteID,clusterID,connectionTime,disconnectTime,kWhDelivered,doneChargingTime,...,doneChargingTimeDay,userID,userID,milesRequested,WhPerMile,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,kWhRequested
0,5bc9160af9af8b0dad3c0690,2_39_95_27_2018-05-01 13:23:18.374684,2-39-95-27,CA-319,2,39,2018-05-01 13:23:18+00:00,2018-05-02 01:53:41+00:00,13.410,2018-05-01 17:17:07+00:00,...,1.0,None,NaN,NaN,NaN,NaN,None,None,None,NaN
1,5bc9160af9af8b0dad3c0691,2_39_79_380_2018-05-01 13:54:33.599590,2-39-79-380,CA-489,2,39,2018-05-01 13:54:34+00:00,2018-05-01 23:15:35+00:00,8.618,2018-05-01 15:35:39+00:00,...,1.0,None,NaN,NaN,NaN,NaN,None,None,None,NaN
2,5bc9160af9af8b0dad3c0692,2_39_139_28_2018-05-01 14:46:04.441389,2-39-139-28,CA-303,2,39,2018-05-01 14:46:04+00:00,2018-05-02 00:54:07+00:00,13.969,2018-05-01 23:57:02+00:00,...,1.0,None,NaN,NaN,NaN,NaN,None,None,None,NaN
3,5bc9160af9af8b0dad3c0693,2_39_79_378_2018-05-01 14:56:28.908623,2-39-79-378,CA-326,2,39,2018-05-01 14:56:29+00:00,2018-05-01 21:52:45+00:00,6.914,2018-05-01 16:07:56+00:00,...,1.0,None,NaN,NaN,NaN,NaN,None,None,None,NaN
4,5bc9160af9af8b0dad3c0694,2_39_88_24_2018-05-01 15:14:50.443137,2-39-88-24,CA-314,2,39,2018-05-01 15:14:50+00:00,2018-05-01 22:51:30+00:00,13.785,2018-05-01 20:18:12+00:00,...,1.0,None,NaN,NaN,NaN,NaN,None,None,None,NaN


In [5]:
start = '2018-05-01'
end = '2021-09-13'
spaceID = 'CA-303'
caiso_fp = 'data/caiso_dataset_5min.csv'
sun_fp = 'data/ct_sr_ss.csv'

X = pd.DataFrame()
sites_df = [df_o,df_c,df_j]
site_ids = [2,1,19]
caiso_fp = 'data/caiso_dataset_5min.csv'


for sdf, id, sun_fp in zip(sites_df, site_ids, ['data/of_sr_ss.csv','data/ct_sr_ss.csv', 'data/jpl_sr_ss.csv'] ):
    start, end = get_start_end_times(sdf)
    tmpx = create_x(start, end, caiso_fp, sun_fp)
    tmpx = update_varuns_x(tmpx, id)
    
    if X.empty:
        X = tmpx
    else:
        X = pd.concat([X, tmpx], axis=0)


X.head()

/home/ewillard/school/capstone/ev_charging/src/data_preprocessing.py:199: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sun['sunset_ts'] = pd.to_datetime(sun['date'] + ' ' + sun['sunset'], errors='coerce', utc=True)
/home/ewillard/school/capstone/ev_charging/src/data_preprocessing.py:199: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sun['sunset_ts'] = pd.to_datetime(sun['date'] + ' ' + sun['sunset'], errors='coerce', utc=True)
/home/ewillard/school/capstone/ev_charging/src/data_preprocessing.py:199: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sun['sunset_ts'] = pd.to_dat

,dow,hour,month,is_sunny,is_holiday,Day ahead forecast,Hour ahead forecast,Current demand,Net demand,Solar,...,Batteries,Imports,Other,Biogas CO2,Biomass CO2,Natural Gas CO2,Coal CO2,Imports CO2,Geothermal CO2,siteID
2019-03-25 00:00:00+00:00,0,0,3,0,False,20066.166667,19685.500000,19634.166667,19247.833333,0.0,...,-4.750000,6223.250000,0.0,124.250000,146.166667,2134.166667,11.750000,2683.250000,9.0,2
2019-03-25 01:00:00+00:00,0,1,3,0,False,19304.500000,18987.166667,18911.583333,18614.416667,0.0,...,-9.500000,5998.916667,0.0,124.166667,145.000000,2278.250000,11.583333,2592.333333,9.0,2
2019-03-25 02:00:00+00:00,0,2,3,0,False,18920.250000,18584.000000,18545.833333,18334.750000,0.0,...,-1.916667,5893.083333,0.0,123.833333,145.750000,2286.250000,11.833333,2535.250000,9.0,2
2019-03-25 03:00:00+00:00,0,3,3,0,False,18858.916667,18575.083333,18539.000000,18330.750000,0.0,...,-2.833333,5701.583333,0.0,123.416667,144.583333,2332.500000,11.833333,2446.333333,9.0,2
2019-03-25 04:00:00+00:00,0,4,3,0,False,19390.416667,19139.666667,19138.500000,18940.916667,0.0,...,-29.833333,5832.083333,0.0,122.833333,144.250000,2447.166667,11.583333,2500.583333,9.0,2


In [6]:
x = X[X.index < '2020-03-01']
x.index.max()
cols = list(df_c.columns)
df = pd.concat([df[cols] for df in sites_df], axis=0)
y = create_all_site_y(df, regression=True)
y = y[y.index < '2020-03-01']
y.index.max()
assert x.shape[0] == y.shape[0], 'forgot to trim y\'s covid!'

y.head()

2019-03-25 00:00:00+00:00    1.0
2019-03-25 01:00:00+00:00    1.0
2019-03-25 02:00:00+00:00    1.0
2019-03-25 03:00:00+00:00    1.0
2019-03-25 04:00:00+00:00    1.0
dtype: float64

In [7]:

import keras
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from keras import layers

2024-03-18 10:04:38.557172: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 10:04:39.534034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


In [13]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    for _ in range(num_transformer_blocks):  
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="elu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x) 
    return keras.Model(inputs, outputs)



In [14]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 6

past = 720
future = 72
learning_rate = 0.001
batch_size = 256
epochs = 10


sequence_length = int(past / step)

X_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

cat_cols = ['dow', 'hour', 'month', 'is_sunny', 'is_holiday','siteID']
num_cols = [col for col in x.columns if col not in cat_cols]

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')
X_train_1 = pd.DataFrame(numeric_transformer.fit_transform(X_train[num_cols]))
X_train_2 = pd.DataFrame(categorical_transformer.fit_transform(X_train[cat_cols]))
X_train_enc = pd.concat([X_train_1, X_train_2], axis=1)
X_train_enc = np.reshape(X_train_enc.values.astype(float), (X_train_enc.shape[0], 1, X_train_enc.shape[1]))

input_shape =(1, X_train_enc.shape[2])
n_classes = len(np.unique(y_train))
print(input_shape)
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#     metrics=["sparse_categorical_accuracy"],
# )
model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["mean_squared_error"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
model.fit(
    X_train_enc,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
)



(1, 67)


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 1, 67)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 67)     │    277,571 │ input_layer_3[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_40          │ (None, 1, 67)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 67)     │        134 │ dropout_40[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 1, 67)     │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_24 (Conv1D)  │ (None, 1, 4)      │        272 │ add_24[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_41          │ (None, 1, 4)      │          0 │ conv1d_24[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_25 (Conv1D)  │ (None, 1, 67)     │        335 │ dropout_41[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 67)     │        134 │ conv1d_25[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_25 (Add)        │ (None, 1, 67)     │          0 │ layer_normalizat… │
│                     │                   │            │ add_24[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 67)     │    277,571 │ add_25[0][0],     │
│ (MultiHeadAttentio… │                   │            │ add_25[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_43          │ (None, 1, 67)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 67)     │        134 │ dropout_43[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_26 (Add)        │ (None, 1, 67)     │          0 │ layer_normalizat… │
│                     │                   │            │ add_25[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_26 (Conv1D)  │ (None, 1, 4)      │        272 │ add_26[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_44          │ (None, 1, 4)      │          0 │ conv1d_26[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_27 (Conv1D)  │ (None, 1, 67)     │        335 │ dropout_44[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 67)     │        134 │ conv1d_27[0][0]   │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 1,114,169 (4.25 MB)

 Trainable params: 1,114,169 (4.25 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - loss: 0.4614 - mean_squared_error: 0.4614 - val_loss: 0.0615 - val_mean_squared_error: 0.0615
Epoch 2/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0689 - mean_squared_error: 0.0689 - val_loss: 0.0380 - val_mean_squared_error: 0.0380
Epoch 3/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0509 - mean_squared_error: 0.0509 - val_loss: 0.0322 - val_mean_squared_error: 0.0323
Epoch 4/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0427 - mean_squared_error: 0.0427 - val_loss: 0.0303 - val_mean_squared_error: 0.0303
Epoch 5/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0397 - mean_squared_error: 0.0397 - val_loss: 0.0253 - val_mean_squared_error: 0.0254
Epoch 6/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0359 - mean_squared_error: 0.0359 - val_loss: 0.0249 - val_mean_squared_error: 0.0250
Epoch 7/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0328 - mean_squared_error: 0.0328 - val_lo

In [16]:
X_test_1 = pd.DataFrame(numeric_transformer.transform(X_test[num_cols]))
X_test_2 = pd.DataFrame(categorical_transformer.transform(X_test[cat_cols]))
X_test_enc = pd.concat([X_test_1, X_test_2], axis=1)
X_test_enc = np.reshape(X_test_enc.values.astype(float), (X_test_enc.shape[0], 1, X_test_enc.shape[1]))
model.evaluate(X_test_enc, y_test, verbose=1)

234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0610 - mean_squared_error: 0.0610


[0.058522842824459076, 0.05858376994729042]